In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [40]:
temp = b'32'

In [41]:
tf.train.Feature(bytes_list=tf.train.BytesList(value=[temp]))

bytes_list {
  value: "32"
}

In [70]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [71]:
temp = {'a': tf.train.Feature(int64_list=tf.train.Int64List(value=[False]))}

In [72]:
temp

{'a': int64_list {
   value: 0
 }}

In [73]:
temp = tf.train.Example(features=tf.train.Features(feature=temp))

In [74]:
temp.SerializeToString()

b'\n\x0c\n\n\n\x01a\x12\x05\x1a\x03\n\x01\x00'

In [91]:
temp = [0, 0, 0]

In [92]:
temp = tf.cast(temp, dtype=tf.float32)

In [93]:
temp

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

In [94]:
tf.reshape(temp, ())

InvalidArgumentError: Input to reshape is a tensor with 3 values, but the requested shape has 1 [Op:Reshape]